<a href="https://colab.research.google.com/github/muditsatija08/genetic-evolution/blob/main/Copy_of_CS6271_EvolutionaryAlgorithmsProject_2023_23256087_23158018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<img src="https://www.ul.ie/themes/custom/ul/logo.jpg" />
</div>

#**MSc in Artificial Intelligence and Machine Learning**
##CS6271 - Evolutionary Algorithms and Humanoid Robotics 2023
### Kaggle Competition


Module Leader: Conor Ryan

Developer: Allan De Lima

Link to access the competition: https://www.kaggle.com/competitions/cs6271-20234-final-project

Link to join the competition: https://www.kaggle.com/t/2b316ba38c144f23ac780c8fc898b4d7



## Introduction

Predict whether income exceeds $50K/yr based on census data. This is a shorter version of the also known as "Census Income" dataset (donated on 4/30/1996).

In [ ]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Dataset

Class:

income: >50K, <=50K.


Listing of features:

age: continuous.

workclass: categorical (Private, Self-emp-not-inc, Local-gov, State-gov).

education: categorical (Bachelors, Some-college, HS-grad, Masters, Doctorate).

marital-status: categorical (Married-civ-spouse, Divorced, Never-married).

relationship: categorical (Wife, Husband, Not-in-family, Other-relative).

race: categorical (White, Asian-Pac-Islander, Black).

sex: categorical (Female, Male).

capital-gain: continuous.

capital-loss: continuous.

hours-per-week: continuous.

native-country: categorical (United-States, Others).


### Load the dataset

In [ ]:
# Suppressing Warnings:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
## mount your Google drive
# 1) run this cell
# 2) sign in
# 3) verify your drive is mounted

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Clone the GRAPE repository at first because the dataset to be used is already there.

In [ ]:
import os
# Get the library from our BDS research Group
# copy the path from your drive
PATH = '/content/drive/MyDrive/grape/'

# check if 'grape' already exists
if os.path.exists(PATH):
    print('grape directory already exists')
else:
    %cd /content/drive/MyDrive/
    !git clone https://github.com/bdsul/grape.git
    print('Cloning grape in your Drive')

# change directory to 'grape'
%cd /content/drive/MyDrive/grape/

grape directory already exists
/content/drive/MyDrive/grape


Now you have a grape folder in your Drive account.

Upload the files adult_training.csv and adult_test.csv to the folder grape/datasets in your Drive before running the next cells.

### Train set

In [ ]:
train_file = 'datasets/adult_training.csv'

In [ ]:
# load train set
df_train = pd.read_csv(PATH+train_file)
df_train.head()

,age,workclass,education,marital-status,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,28,Private,Bachelors,Never-married,Not-in-family,White,Male,0,0,40,United-States,<=50K
1,34,Self-emp-not-inc,Bachelors,Married-civ-spouse,Husband,Black,Male,0,1887,48,United-States,>50K
2,32,Private,Bachelors,Never-married,Not-in-family,Black,Female,0,0,40,United-States,<=50K
3,46,Private,Bachelors,Divorced,Not-in-family,White,Male,0,0,40,Others,<=50K
4,44,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,0,50,United-States,>50K


In [ ]:
df_train.describe()

,age,capital-gain,capital-loss,hours-per-week
count,5200.000000,5200.00000,5200.000000,5200.000000
mean,39.688077,1059.89500,109.486346,42.786538
std,11.973363,6687.36408,442.694051,10.937644
min,17.000000,0.00000,0.000000,1.000000
25%,30.000000,0.00000,0.000000,40.000000
50%,38.000000,0.00000,0.000000,40.000000
75%,48.000000,0.00000,0.000000,48.000000
max,90.000000,99999.00000,2559.000000,99.000000


In [ ]:
X_train = df_train.copy()
# warning: cannot drop it more than once
X_train.drop(['income'], axis=1, inplace=True)

You should represent the outputs with 0 where the income is smaller or equal to 50K and with 1 if it is greater than 50K.

Follow exactly this approach, because the test targets are represented like this in the competition.

In [ ]:
# class labels
l, _ = X_train.shape

y_train = np.zeros([l,], dtype=int)

for i in range(l):
  if df_train['income'].iloc[i] == '>50K':
    y_train[i] = 1
  elif df_train['income'].iloc[i] == '<=50K':
    y_train[i] = 0

In [ ]:
print(y_train[0:5]) #print head

[0 1 0 0 1]


### Test set

In [ ]:
test_file = 'datasets/adult_test.csv'

In [ ]:
# load test set
df_test = pd.read_csv(PATH+test_file)
df_test.head()

,age,workclass,education,marital-status,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,33,Private,HS-grad,Never-married,Not-in-family,White,Male,3325,0,50,United-States
1,58,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States
2,30,Self-emp-not-inc,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,60,United-States
3,26,Private,Some-college,Never-married,Not-in-family,White,Female,0,0,20,United-States
4,43,State-gov,HS-grad,Never-married,Not-in-family,White,Male,0,0,60,United-States


In [ ]:
df_test.describe()

,age,capital-gain,capital-loss,hours-per-week
count,10402.000000,10402.000000,10402.000000,10402.000000
mean,39.811575,1280.969237,106.101038,42.749567
std,12.063746,7826.438595,438.826968,11.200949
min,18.000000,0.000000,0.000000,1.000000
25%,30.000000,0.000000,0.000000,40.000000
50%,38.000000,0.000000,0.000000,40.000000
75%,48.000000,0.000000,0.000000,48.000000
max,90.000000,99999.000000,3683.000000,99.000000


In [ ]:
X_test = df_test.copy()

You will need to prepare both training and test datasets before working with a Machine Learning method.

Consider you need to use some encoding method with categorical data.

You are free to use any other pre-processing ideas.

In [ ]:
X_train

,age,workclass,education,marital-status,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,28,Private,Bachelors,Never-married,Not-in-family,White,Male,0,0,40,United-States
1,34,Self-emp-not-inc,Bachelors,Married-civ-spouse,Husband,Black,Male,0,1887,48,United-States
2,32,Private,Bachelors,Never-married,Not-in-family,Black,Female,0,0,40,United-States
3,46,Private,Bachelors,Divorced,Not-in-family,White,Male,0,0,40,Others
4,44,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,0,50,United-States
...,...,...,...,...,...,...,...,...,...,...,...
5195,57,Private,Masters,Divorced,Not-in-family,White,Female,0,0,40,United-States
5196,23,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,0,40,United-States
5197,47,Private,Masters,Married-civ-spouse,Husband,White,Male,0,0,65,United-States
5198,66,Private,Bachelors,Married-civ-spouse,Husband,White,Male,20051,0,40,United-States


In [ ]:
y_train

array([0, 1, 0, ..., 1, 1, 0])

In [ ]:
X_test

,age,workclass,education,marital-status,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,33,Private,HS-grad,Never-married,Not-in-family,White,Male,3325,0,50,United-States
1,58,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States
2,30,Self-emp-not-inc,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,60,United-States
3,26,Private,Some-college,Never-married,Not-in-family,White,Female,0,0,20,United-States
4,43,State-gov,HS-grad,Never-married,Not-in-family,White,Male,0,0,60,United-States
...,...,...,...,...,...,...,...,...,...,...,...
10397,30,Private,Some-college,Married-civ-spouse,Husband,White,Male,0,0,45,United-States
10398,28,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,0,60,United-States
10399,40,Private,Some-college,Married-civ-spouse,Husband,White,Male,0,0,60,United-States
10400,40,Local-gov,Some-college,Married-civ-spouse,Husband,White,Male,0,0,42,United-States


In [ ]:
# Categorical columns for one-hot encoding
categorical_cols = ['workclass', 'education', 'marital-status', 'relationship', 'race', 'sex', 'native-country']

# # One-hot encode categorical variables
X_train = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)

# # Convert income labels to binary (0 or 1)
# df_train['income'] = df_train['income'].map({'>50K': 1, '<=50K': 0})
# df_test['income'] = df_test['income'].map({'>50K': 1, '<=50K': 0})

# # Separate features and target variable in the training set
# X_train = df_train.drop('income', axis=1)
# y_train = df_train['income']

# Separate features in the test set
# X_test = df_test

In [ ]:
X_train

,age,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Self-emp-not-inc,workclass_State-gov,education_Doctorate,education_HS-grad,education_Masters,education_Some-college,marital-status_Married-civ-spouse,marital-status_Never-married,relationship_Not-in-family,relationship_Other-relative,relationship_Wife,race_Black,race_White,sex_Male,native-country_United-States
0,28,0,0,40,1,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1
1,34,0,1887,48,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,1
2,32,0,0,40,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1
3,46,0,0,40,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0
4,44,0,0,50,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,57,0,0,40,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1
5196,23,0,0,40,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1
5197,47,0,0,65,1,0,0,0,0,1,0,1,0,0,0,0,0,1,1,1
5198,66,20051,0,40,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1


In [ ]:
X_test

,age,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Self-emp-not-inc,workclass_State-gov,education_Doctorate,education_HS-grad,education_Masters,education_Some-college,marital-status_Married-civ-spouse,marital-status_Never-married,relationship_Not-in-family,relationship_Other-relative,relationship_Wife,race_Black,race_White,sex_Male,native-country_United-States
0,33,3325,0,50,1,0,0,0,1,0,0,0,1,1,0,0,0,1,1,1
1,58,0,0,40,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,1
2,30,0,0,60,0,1,0,0,1,0,0,1,0,0,0,0,0,1,1,1
3,26,0,0,20,1,0,0,0,0,0,1,0,1,1,0,0,0,1,0,1
4,43,0,0,60,0,0,1,0,1,0,0,0,1,1,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10397,30,0,0,45,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,1
10398,28,0,0,60,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1
10399,40,0,0,60,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,1
10400,40,0,0,42,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,1


In [ ]:
#Include your code here
from sklearn.preprocessing import StandardScaler

In [ ]:
# Normalizing the numeric data
# X_train.loc[:, ["age", "capital-gain", "capital-loss", "hours-per-week"]] = (X_train.loc[:, ["age", "capital-gain", "capital-loss", "hours-per-week"]] - X_train.loc[:, ["age", "capital-gain", "capital-loss", "hours-per-week"]].mean()) / X_train.loc[:, ["age", "capital-gain", "capital-loss", "hours-per-week"]].std()
# X_test.loc[:, ["age", "capital-gain", "capital-loss", "hours-per-week"]] = (X_test.loc[:, ["age", "capital-gain", "capital-loss", "hours-per-week"]] - X_test.loc[:, ["age", "capital-gain", "capital-loss", "hours-per-week"]].mean()) / X_test.loc[:, ["age", "capital-gain", "capital-loss", "hours-per-week"]].std()

sc = StandardScaler()
column_names = ["age", "capital-gain", "capital-loss", "hours-per-week"]
X_train[column_names] = sc.fit_transform(X_train[column_names])
X_test[column_names] = sc.transform(X_test[column_names])

In [ ]:
X_train

,age,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Self-emp-not-inc,workclass_State-gov,education_Doctorate,education_HS-grad,education_Masters,education_Some-college,marital-status_Married-civ-spouse,marital-status_Never-married,relationship_Not-in-family,relationship_Other-relative,relationship_Wife,race_Black,race_White,sex_Male,native-country_United-States
0,-0.976267,-0.158507,-0.247342,-0.254790,1,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1
1,-0.475107,-0.158507,4.015605,0.476699,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,1
2,-0.642160,-0.158507,-0.247342,-0.254790,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1
3,0.527214,-0.158507,-0.247342,-0.254790,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0
4,0.360161,-0.158507,-0.247342,0.659571,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,1.446009,-0.158507,-0.247342,-0.254790,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1
5196,-1.393901,-0.158507,-0.247342,-0.254790,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1
5197,0.610741,-0.158507,-0.247342,2.031114,1,0,0,0,0,1,0,1,0,0,0,0,0,1,1,1
5198,2.197750,2.840122,-0.247342,-0.254790,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1


In [ ]:
X_test

,age,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Self-emp-not-inc,workclass_State-gov,education_Doctorate,education_HS-grad,education_Masters,education_Some-college,marital-status_Married-civ-spouse,marital-status_Never-married,relationship_Not-in-family,relationship_Other-relative,relationship_Wife,race_Black,race_White,sex_Male,native-country_United-States
0,-0.558633,0.338747,-0.247342,0.659571,1,0,0,0,1,0,0,0,1,1,0,0,0,1,1,1
1,1.529535,-0.158507,-0.247342,-0.254790,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,1
2,-0.809214,-0.158507,-0.247342,1.573933,0,1,0,0,1,0,0,1,0,0,0,0,0,1,1,1
3,-1.143321,-0.158507,-0.247342,-2.083514,1,0,0,0,0,0,1,0,1,1,0,0,0,1,0,1
4,0.276634,-0.158507,-0.247342,1.573933,0,0,1,0,1,0,0,0,1,1,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10397,-0.809214,-0.158507,-0.247342,0.202390,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,1
10398,-0.976267,-0.158507,-0.247342,1.573933,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1
10399,0.026054,-0.158507,-0.247342,1.573933,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,1
10400,0.026054,-0.158507,-0.247342,-0.071918,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,1


Convert the datasets to NumPy to easily use them.

In [ ]:

# data features
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

In [ ]:
X_train

array([[-0.97626713, -0.15850744, -0.2473421 , ...,  1.        ,
         1.        ,  1.        ],
       [-0.4751066 , -0.15850744,  4.01560533, ...,  0.        ,
         1.        ,  1.        ],
       [-0.64216011, -0.15850744, -0.2473421 , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.6107412 , -0.15850744, -0.2473421 , ...,  1.        ,
         1.        ,  1.        ],
       [ 2.19774953,  2.84012222, -0.2473421 , ...,  1.        ,
         1.        ,  1.        ],
       [ 0.10958068, -0.15850744, -0.2473421 , ...,  1.        ,
         1.        ,  1.        ]])

In [ ]:
X_test

array([[-0.55863336,  0.33874675, -0.2473421 , ...,  1.        ,
         1.        ,  1.        ],
       [ 1.5295355 , -0.15850744, -0.2473421 , ...,  1.        ,
         1.        ,  1.        ],
       [-0.80921362, -0.15850744, -0.2473421 , ...,  1.        ,
         1.        ,  1.        ],
       ...,
       [ 0.02605392, -0.15850744, -0.2473421 , ...,  1.        ,
         1.        ,  1.        ],
       [ 0.02605392, -0.15850744, -0.2473421 , ...,  1.        ,
         1.        ,  1.        ],
       [ 1.11190173, -0.15850744, -0.2473421 , ...,  1.        ,
         0.        ,  1.        ]])

## GRAPE

<div>
<img src="https://drive.google.com/uc?export=view&id=1hw43Oi3lGTCkspQ0ged2bZB8q2EpcPhz" width="150"/>
</div>

GRammatical Algorithms in Python for Evolution (GRAPE)


In [ ]:
!pip install deap

import grape
import algorithms

from os import path
from deap import creator, base, tools
import random
import csv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.9 MB/s eta 0:00:00


You can import functions to be used with your grammar from [functions.py](https://github.com/UL-BDS/grape/blob/main/functions.py) on GRAPE repository and / or you can define your own functions.

In [ ]:
from functions import add, sub, mul, pdiv, psqrt, plog, and_, or_, not_, less_than_or_equal, greater_than_or_equal

'heartDisease.bnf' is a grammar used for another problem just to check if everything is working well.

Write your own grammar in a text file and save it in your Drive account.

Put the whole address on GRAMMAR_FILE and print to check it.

In [ ]:
GRAMMAR_FILE = '/content/drive/MyDrive/data/mygrammar.bnf' #put the whole address of your own grammar and remove the # in the beginning of this line
# GRAMMAR_FILE = 'heartDisease.bnf' #remove this line when you are using your own grammar

f = open(GRAMMAR_FILE, "r") #remove the # in the beginning of this line when you are using your own grammar
# f = open("grammars/" + GRAMMAR_FILE, "r") #remove this line when you are using your own grammar
print(f.read())
f.close()


<log_op> ::= <conditional_branches> | and_(<log_op>,<log_op>) | or_(<log_op>,<log_op>) | not_(<log_op>) | <boolean_feature>
<conditional_branches> ::= less_than_or_equal(<num_op>,<num_op>) | greater_than_or_equal(<num_op>, <num_op>)
<num_op>   ::= add(<num_op>,<num_op>) | sub(<num_op>,<num_op>) | mul(<num_op>,<num_op>) | pdiv(<num_op>,<num_op>) | <nonboolean_feature>
<boolean_feature> ::= x[4]|x[5]|x[6]|x[7]|x[8]|x[9]|x[10]|x[11]|x[12]|x[13]|x[14]|x[15]|x[16]|x[17]|x[18]|x[19]
<nonboolean_feature> ::= x[0]|x[1]|x[2]|x[3]
 


Run the following cell to put your grammar on the class Grammar.

In [ ]:
BNF_GRAMMAR = grape.Grammar(GRAMMAR_FILE) #remove the # in the beginning of this line when you are using your own grammar
# BNF_GRAMMAR = grape.Grammar(path.join("grammars", GRAMMAR_FILE)) #remove this line when you are using your own grammar

The fitness function here is the percentage of outputs wrongly predicted.

You can write your own fitness function if you prefer.

In [ ]:
def fitness_eval(individual, points):
    """
    Fitness Function
    """

    x = points[0]
    Y = points[1]

    if individual.invalid == True:
        return np.NaN,

    # Evaluate the expression
    try:
        pred = eval(individual.phenotype)
    except (FloatingPointError, ZeroDivisionError, OverflowError,
            MemoryError):
        return np.NaN,
    assert np.isrealobj(pred)

    compare = np.equal(Y,pred)
    fitness = 1 - np.mean(compare)

    return fitness,

To use properly the fitness function above with GRAPE, the features must be in the lines, and the samples must be in the columns, so if your data is not like that, you need to transpose the matrix.

Take a look at the print. If you run this cell two times, the matrix will be transposed again and will not work properly.

In [ ]:
X_train = np.transpose(X_train)
X_test = np.transpose(X_test)

print('Training (X,Y):\t', X_train.shape, y_train.shape)
print('Test (X):\t', X_test.shape)

Training (X,Y):	 (20, 5200) (5200,)
Test (X):	 (20, 10402)


Set the Grammatical Evolution parameters.

Make sure you set a random seed just in case we need to re-run your experiments.

In [ ]:
POPULATION_SIZE = 900
MAX_GENERATIONS = 150
P_CROSSOVER = 0.88
P_MUTATION = 0.045
ELITE_SIZE = 10
HALL_OF_FAME_SIZE = 50

TOURNAMENT_SIZE = 15
RANDOM_SEED = 412
random.seed(RANDOM_SEED)

CODON_CONSUMPTION = 'lazy'
GENOME_REPRESENTATION = 'list'
MAX_GENOME_LENGTH = None

MAX_INIT_TREE_DEPTH = 12
MIN_INIT_TREE_DEPTH = 4
MAX_TREE_DEPTH = 12
MAX_WRAPS = 0
CODON_SIZE = 255

REPORT_ITEMS = ['gen', 'invalid', 'avg', 'std', 'min', 'max',
                'best_ind_length', 'avg_length',
                'best_ind_nodes', 'avg_nodes',
                'best_ind_depth', 'avg_depth',
                'avg_used_codons', 'best_ind_used_codons',
                'structural_diversity', 'fitness_diversity',
                'selection_time', 'generation_time']

Create a toolbox.

In [ ]:
toolbox = base.Toolbox()

# define a single objective, minimising fitness strategy:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

creator.create('Individual', grape.Individual, fitness=creator.FitnessMin)

toolbox.register("populationCreator", grape.sensible_initialisation, creator.Individual)

toolbox.register("evaluate", fitness_eval)

# Tournament selection:
toolbox.register("select", tools.selTournament, tournsize=TOURNAMENT_SIZE)

# Single-point crossover:
toolbox.register("mate", grape.crossover_onepoint)

# Flip-int mutation:
toolbox.register("mutate", grape.mutation_int_flip_per_codon)

In [ ]:
# create initial population (generation 0):
population = toolbox.populationCreator(pop_size=POPULATION_SIZE,
                                           bnf_grammar=BNF_GRAMMAR,
                                           min_init_depth=MIN_INIT_TREE_DEPTH,
                                           max_init_depth=MAX_INIT_TREE_DEPTH,
                                           codon_size=CODON_SIZE,
                                           codon_consumption=CODON_CONSUMPTION,
                                           genome_representation=GENOME_REPRESENTATION
                                            )

# define the hall-of-fame object:
hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

# prepare the statistics object:
stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.nanmean)
stats.register("std", np.nanstd)
stats.register("min", np.nanmin)
stats.register("max", np.nanmax)

Run Grammatical Evolution.

In [ ]:
population, logbook = algorithms.ge_eaSimpleWithElitism(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION,
                                              ngen=MAX_GENERATIONS, elite_size=ELITE_SIZE,
                                              bnf_grammar=BNF_GRAMMAR,
                                              codon_size=CODON_SIZE,
                                              max_tree_depth=MAX_TREE_DEPTH,
                                              max_genome_length=MAX_GENOME_LENGTH,
                                              points_train=[X_train, y_train],
                                              codon_consumption=CODON_CONSUMPTION,
                                              report_items=REPORT_ITEMS,
                                              genome_representation=GENOME_REPRESENTATION,
                                              stats=stats, halloffame=hof, verbose=False)

gen = 0 , Best fitness = (0.30153846153846153,)
gen = 1 , Best fitness = (0.30153846153846153,) , Number of invalids = 136
gen = 2 , Best fitness = (0.30153846153846153,) , Number of invalids = 100
gen = 3 , Best fitness = (0.3011538461538461,) , Number of invalids = 156
gen = 4 , Best fitness = (0.2809615384615385,) , Number of invalids = 134
gen = 5 , Best fitness = (0.2809615384615385,) , Number of invalids = 113
gen = 6 , Best fitness = (0.2798076923076923,) , Number of invalids = 96
gen = 7 , Best fitness = (0.2798076923076923,) , Number of invalids = 68
gen = 8 , Best fitness = (0.2798076923076923,) , Number of invalids = 57
gen = 9 , Best fitness = (0.2798076923076923,) , Number of invalids = 37
gen = 10 , Best fitness = (0.27749999999999997,) , Number of invalids = 20
gen = 11 , Best fitness = (0.27749999999999997,) , Number of invalids = 21
gen = 12 , Best fitness = (0.27749999999999997,) , Number of invalids = 18
gen = 13 , Best fitness = (0.27749999999999997,) , Number of in

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-75-94415cdce62f>", line 1, in <cell line: 1>
    population, logbook = algorithms.ge_eaSimpleWithElitism(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION,
  File "/content/drive/MyDrive/grape/algorithms.py", line 224, in ge_eaSimpleWithElitism
    offspring = varAnd(offspring, toolbox, cxpb, mutpb,
  File "/content/drive/MyDrive/grape/algorithms.py", line 41, in varAnd
    offspring = [toolbox.clone(ind) for ind in population]
  File "/content/drive/MyDrive/grape/algorithms.py", line 41, in <listcomp>
    offspring = [toolbox.clone(ind) for ind in population]
  File "/usr/lib/python3.10/copy.py", line 172, in deepcopy
    y = _reconstruct(x, memo, *rv)
  File "/usr/lib/python3.10/copy.py", line 271, in _reconstruct
    state = deepcopy(state, memo)
  File "/usr/lib

Show the best individual as an expression.

In [ ]:
# Best individual
import textwrap
best = hof.items[0].phenotype
print("Best individual: \n","\n".join(textwrap.wrap(best,80)))
print("\nTraining Fitness: ", hof.items[0].fitness.values[0])
print("Depth: ", hof.items[0].depth)
print("Length of the genome: ", len(hof.items[0].genome))
print(f'Used portion of the genome: {hof.items[0].used_codons/len(hof.items[0].genome):.2f}')

Define a function to predict values, without comparing to expected outputs.

In [ ]:
def predict(individual, X):
    x = X

    if individual.invalid == True:
        return np.NaN,

    # Evaluate the expression
    try:
        pred = eval(individual.phenotype)
    except (FloatingPointError, ZeroDivisionError, OverflowError,
            MemoryError):
        return np.NaN,
    assert np.isrealobj(pred)

    return pred

Predict the classes of the test set.

Make sure you print here in the notebook you will submit to Brightspace the same predictions you used in your best submission to the Kaggle competition.

In [ ]:
y_pred = predict(hof.items[0], X_test)
print("Predicted classes of the test set: ", y_pred)

Write a code to create a .csv with the following format:
1. First column is the index (from 0 to 10401);
2. Second column is named `income` and contains the predictions (only 0's or 1's) you  got in the previous cell with y_pred.

Example:

    index,income

    0,0

    1,0

    2,1

    ...

    10401,0


Submit it to the competition and check your score there.

In [ ]:
from operator import index
data = {'index': np.arange(10402),
        'income':y_pred}

df = pd.DataFrame(data)

df.to_csv('predictions.csv', index = False)

In [ ]:
df

,index,income
0,0,False
1,1,False
2,2,False
3,3,False
4,4,False
...,...,...
10397,10397,False
10398,10398,True
10399,10399,False
10400,10400,False
